In [1]:
# system import
import sys
import os
import os.path
import csv
import config
import random
import time

# data processing
import pandas as pd


# data Extraction
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

In [2]:
# set env
PROJECT_PATH = config.data_directory
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# try to not get banned
USER_AGENTS = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Mozilla/5.0 (iPad; CPU OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H321 Safari/600.1.4',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:11.0) Gecko/20100101 Firefox/11.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36 OPR/58.0.3135.107']

def getFromUrl(url, user_agent=True):
    time.sleep(random.randint(1,10))
    if user_agent:
        return requests.get(url, headers={'User-Agent': random.choice(USER_AGENTS)})
    return requests.get(url)

def getLyric(title):
    url = "https://www.azlyrics.com/lyrics/{}.html".format(title)
    return getFromUrl(url)

def seachLyric(title, artist):
    url = "https://search.azlyrics.com/search.php?q={}".format(title.replace(' ', '+').lower())
    c = getFromUrl(url)
    soup = BeautifulSoup(c.content)
    links = soup.find_all('b', string=artist)
    if len(links) > 0:
        link = links[0].find_previous_sibling('a')['href']
        return getFromUrl(link)
    else:
        return None

In [3]:
# file operations
def read_file(file):
    text_file = open('{}'.format(file), "r", encoding='utf-8')
    text = text_file.read()
    text_file.close()
    return text

def get_lyric_name_path(a, t):
    aaz = a.replace(' ', '').lower()
    taz = t.replace(' ', '').lower()
    title = '{}/{}'.format(aaz, taz)
    # print(title)
    file_path = '{}/{}.txt'.format(PROJECT_PATH, title.replace('/','-'))
    path = [title, file_path]
    
    if os.path.isfile(file_path):
        return path + [True]
    else:
        return path + [False]

def fetch_lylic_to_file(a, t, aa = None, tt = None):
    path = get_lyric_name_path(a, t)

    print(path)
    if not path[2]:
        result = getLyric(path[0])
        
        if result.status_code != requests.codes.ok:
            result = seachLyric(t, aa)
        
        if result is None:
            return False
            
        c = result.content
        soup = BeautifulSoup(c)
        ringtones = soup.find_all("div", class_="ringtone")
        
        if len(ringtones) > 0:
            ringtone = ringtones[0]
            html_lyrics = ringtone.find_next_sibling('div')
            cleanText = html_lyrics.get_text().replace('\r','').replace('\n',' ').replace("\'",'"')

            fo = open(path[1], "w", encoding='utf-8')
            fo.write(cleanText)
            fo.close()
            
            return True
        else:
            return False
    else:
        print('File Already Exists')
        return False

In [4]:
def splitChums(target, n = 100):
    x = (len(target) + n - 1) // n
    rg = range(x)
    return [target[i * n:(i + 1) * n] for i in rg]

In [5]:
# spotify interaction
def fetch_songs(offset, count, limit = 50, startYear = 1990, name = 'songs'):
    file_path = '{}/{}.csv'.format(PROJECT_PATH, name)
    new_file = 'a'
    tracks = []
    if not os.path.exists(file_path):
        tracks = [['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artist', 'artist_id', 'album', 'album_id', 'album_release_date', 'album_total_tracks']]
        new_file = 'w'

    partition = (((count * limit) + 10 - 1) // 10)
    with open(file_path, new_file, newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        for i in range(offset, count * limit, limit):
            if i % partition: 
                startYear = startYear + 1
            track_results = sp.search(q='year:' + str(startYear), type='track', limit=limit, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                tracks.append([t['id'], t['name'], t['popularity'], t['duration_ms'], t['explicit'], t['artists'][0]['name'], t['artists'][0]['id'],  t['album']['name'], t['album']['id'], t['album']['release_date'], t['album']['total_tracks']])
        writer.writerows(tracks)
    csvFile.close()

In [6]:
fetch_songs(0, 50)

In [7]:
# load dataFrame
df = pd.read_csv('{}/{}.csv'.format(PROJECT_PATH, 'songs'))

# az data
pattern = r'[^a-z0-9A-z]'
df['p_name'] = df['name'].str.replace(pattern, ' ').str.lower()
df['p_artist'] = df['artist'].str.replace(pattern, ' ').str.lower()
df['file_name'] = df.apply(lambda x: get_lyric_name_path(x['p_artist'], x['p_name'])[1], axis=1)
df['has_lyrics'] = False

In [8]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_total_tracks,p_name,p_artist,file_name,has_lyrics
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,80,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,12,thunderstruck,ac dc,./data/acdc-thunderstruck.txt,False
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,69,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,67,don t go breaking my heart,elton john,./data/eltonjohn-dontgobreakingmyheart.txt,False
2,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,70,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,12,man in the box,alice in chains,./data/aliceinchains-maninthebox.txt,False
3,6m59VvDUi0UQsB2eZ9wVbH,Poison,66,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,10,poison,bell biv devoe,./data/bellbivdevoe-poison.txt,False
4,3Ti0GdlrotgwsAVBBugv0I,Can I Kick It?,69,251573,False,A Tribe Called Quest,09hVIj6vWgoCDtT03h8ZCa,People's Instinctive Travels and the Paths of ...,3kV0i1qqudjf0PGawJ4jck,1990-04-17,17,can i kick it,a tribe called quest,./data/atribecalledquest-canikickit.txt,False


In [9]:
# get audio features
ids = df['id'].tolist()

cIds = splitChums(ids)

features = []

for cId in cIds:
    audio_results = sp.audio_features(cId)
    for af in audio_results:
        if af:
            features.append([af['id'], af['danceability'], af['energy'], af['key'], af['loudness'], af['mode'], af['speechiness'], af['acousticness'], af['instrumentalness'], af['liveness'], af['valence'], af['tempo']])

In [11]:
# get genre features
ids = df['artist_id'].tolist()
ids = list(dict.fromkeys(ids))
cIds = splitChums(ids, 50)
artists = []

for cId in cIds:
    artists_results = sp.artists(cId)
    for ar in artists_results['artists']:
        if ar:
            artists.append([ar['id'], ar['genres']])

In [12]:
dfaf = pd.DataFrame(features, columns = ['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

df = pd.merge(df, dfaf, on='id', how='inner')

In [13]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,80,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,...,0.889,4,-5.175,1,0.0364,0.000147,0.011400,0.2170,0.257,133.519
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,69,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,...,0.858,5,-7.790,1,0.0416,0.196000,0.000011,0.0685,0.779,131.312
2,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,70,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,...,0.765,8,-8.558,1,0.0413,0.002290,0.000008,0.2130,0.644,106.463
3,6m59VvDUi0UQsB2eZ9wVbH,Poison,66,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,...,0.573,6,-10.316,0,0.2240,0.002160,0.004320,0.6120,0.803,111.848
4,3Ti0GdlrotgwsAVBBugv0I,Can I Kick It?,69,251573,False,A Tribe Called Quest,09hVIj6vWgoCDtT03h8ZCa,People's Instinctive Travels and the Paths of ...,3kV0i1qqudjf0PGawJ4jck,1990-04-17,...,0.666,0,-6.547,1,0.2740,0.173000,0.000699,0.1290,0.744,96.662


In [14]:
dfa = pd.DataFrame(artists, columns = ['artist_id', 'genres'])

df = pd.merge(df, dfa, on='artist_id', how='inner')

In [15]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,80,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,...,4,-5.175,1,0.0364,0.000147,0.011400,0.2170,0.257,133.519,"[album rock, australian rock, hard rock, rock]"
1,5u5F7qLDvZjBSktaDp4HxB,Moneytalks,62,225946,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,...,7,-3.677,1,0.0314,0.000841,0.000351,0.1220,0.409,121.896,"[album rock, australian rock, hard rock, rock]"
2,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,69,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,...,5,-7.790,1,0.0416,0.196000,0.000011,0.0685,0.779,131.312,"[glam rock, mellow gold, piano rock, rock, sof..."
3,0EANX0OVKSCcmarY50Xa4p,Can You Feel the Love Tonight - End Title/ Fro...,57,241013,False,Elton John,3PhoLpVuITZKcymswpck5b,The Lion King,3YA5DdB3wSz4pdfEXoMyRd,1994-01-01,...,10,-17.077,1,0.0337,0.880000,0.000346,0.1170,0.167,121.498,"[glam rock, mellow gold, piano rock, rock, sof..."
4,7em38Hh2sKwnZ1HDgxvg82,Circle Of Life,47,290093,False,Elton John,3PhoLpVuITZKcymswpck5b,Love Songs,1bnoqPJyaxMCtYuSprcBTD,1995-11-06,...,10,-8.089,1,0.0369,0.283000,0.440000,0.1070,0.157,160.450,"[glam rock, mellow gold, piano rock, rock, sof..."


In [16]:
# save dataframe to csv file
def dataframe_to_csv(pDf, name = 'songs'):
    file_path = '{}/{}.csv'.format(PROJECT_PATH, name)
    pDf.to_csv(file_path, sep=',', encoding='utf-8', index=False)

In [ ]:
# fetch from internet
for index, row in df.iterrows():
    df.loc[index, 'has_lyrics'] = fetch_lylic_to_file(row['p_artist'], row['p_name'], row['artist'], row['name'])

In [17]:
# fetch from local
for index, row in df.iterrows():
    exists = os.path.isfile(row['file_name'])
    if exists:
        df.loc[index, 'has_lyrics'] = exists

In [18]:
df.tail(50)

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
1852,0jT6M1tT51b1p2s8qDUUkB,Show Me Love,64,211655,False,Above & Beyond,10gzBoINW3cLJfZUka8Zoe,Show Me Love,6CRr9qtC3ApJO9Qd1XKfB2,2019-03-01,...,4,-5.203,0,0.0437,0.022000,0.508000,0.1570,0.454000,128.017,"[edm, progressive house, progressive trance, t..."
1853,75VRabO3e7Twgd7mSoPdbF,Position To Win,61,185022,False,Migos,6oMuImdp5ZcFhWP0ESe6mG,Position To Win,41zs8j0l7i3gWRloMuQ45j,2019-02-08,...,10,-4.632,0,0.2650,0.193000,0.000000,0.4320,0.660000,166.126,"[atl hip hop, pop, pop rap, rap, southern hip ..."
1854,0LcspVKJxhEQQSvVMiTPWz,More Hearts Than Mine,59,214160,False,Ingrid Andress,0jPnVIasXzBYjrlpO5irii,More Hearts Than Mine,4VMYwWFqX9vUv9otWLRRF5,2019-04-05,...,0,-7.414,1,0.0296,0.111000,0.000000,0.1140,0.391000,79.985,[country pop]
1855,3NJcATra6ASZtwFY0vIAER,Next To You (feat. Rvssian),64,179257,False,Becky G,4obzFoKoKRHIphyHzJ35G3,Next To You (feat. Rvssian),5IvFsy6CtVoyI0I8upyZ9b,2019-04-26,...,7,-5.340,1,0.0684,0.308000,0.000028,0.1560,0.229000,105.084,"[latin viral pop, pop, reggaeton]"
1856,4v2BLMhWtVvBnZwuoiKxja,"HALA HALA (Hearts Awakened, Live Alive)",65,202097,False,ATEEZ,68KmkJeZGfwe1OUaivBa2L,TREASURE EP.2: Zero To One,2PQ8cQ8WzAJJkWYT2Ydmhw,2019-01-15,...,9,-4.178,0,0.1890,0.015300,0.000000,0.1880,0.294000,159.904,[k-pop]
1857,6vkZlB6ENDJgiDuR88UlaU,Fat Beat,66,158340,False,Will Sparks,1u7OVFmWah4wQhOPIbUb8U,Fat Beat,20SSnc2sKancuJyksWWovw,2019-03-04,...,6,-6.346,0,0.0565,0.007250,0.688000,0.4750,0.060100,139.738,"[big room, deep big room, edm, electro house, ..."
1858,2PD75SFUXxkcbIL4dg0Xgu,I'm Single,61,161660,True,Jake Paul,41hhOkoRcm7m4fE0g7t1sa,I'm Single,7iO3PKNc5oEIdW6tirp8JW,2019-03-01,...,7,-5.729,0,0.1880,0.255000,0.000000,0.1310,0.621000,93.025,[]
1859,3NLm801woJocONz1NmPJZR,Curls,61,197933,False,Bibio,0qzzGu8qpbXYpzgV52wOFT,Curls,1cAlKAmWKhORglznBK7sup,2019-02-06,...,9,-10.593,1,0.0285,0.879000,0.796000,0.7220,0.227000,82.176,"[ambient folk, chamber pop, chillwave, downtem..."
1860,1W10LzfhWQjeycUgofVdnc,So Far So Fast,56,396893,False,The National,2cCUtGK9sDU2EoElnk0GNB,I Am Easy to Find,240Gfpkjr06NUT3zBYrwVN,2019-05-17,...,4,-11.206,1,0.0380,0.717000,0.456000,0.0645,0.265000,111.558,"[chamber pop, folk-pop, indie folk, indie pop,..."
1861,4ZTX8P6yC1dv6CY7b0Ql3g,Life Is Strange,64,196618,False,MARINA,6CwfuxIqcltXDGjfZsMd9A,LOVE + FEAR,0CUxS3KfHNuDpUUjbAewV3,2019-04-26,...,0,-6.745,1,0.0306,0.240000,0.000000,0.0880,0.374000,116.010,"[dance pop, electropop, metropopolis, pop, pos..."


In [20]:
dataframe_to_csv(df)

In [19]:
df['has_lyrics'].describe()

count     1902
unique       2
top       True
freq      1088
Name: has_lyrics, dtype: object